# Manipulating the data

As people are most familar with the SQL syntax we will begin by explaining how familar SQL operations can be done in pandas. Then we we'll move onto things that SQL can't do (or can't do easily!)

### SQL equivalents

First let's load in the library and the data

In [ ]:
import pandas as pd
working_dir  = "../data/"

household_size = pd.read_csv(working_dir + 'HouseholdSize.csv', encoding = 'ISO-8859-1')
admin_regions = pd.read_csv(working_dir + 'AdminRegions.csv', encoding = 'ISO-8859-1')
approximated_social_grade = pd.read_csv(working_dir + 'ApproximatedSocialGrade.csv', encoding = 'ISO-8859-1')
country_of_birth = pd.read_csv(working_dir + 'CountryOfBirthDetailed.csv', encoding = 'ISO-8859-1')
customer_data = pd.read_csv(working_dir + 'CustomerData.csv', encoding = 'ISO-8859-1')
postcode_to_ward_code = pd.read_csv(working_dir + 'PostcodeToWardCode.csv', encoding = 'ISO-8859-1')
household_lifestage = pd.read_csv(working_dir + 'HouseholdLifestage.csv', encoding = 'ISO-8859-1')

### Selects

Selecting in pandas can be done using a list of column names inside square brackets

In [ ]:
admin_stage = admin_regions[["ward_code", "ward_name", 
            "local_authority_name", "region_name", "country_name"]]
admin_stage.head()

It can also be done using `loc` but note that we need to put a colon in for the first argument. More about `loc` below.

In [ ]:
admin_stage = admin_regions.loc[:, ["ward_code", "ward_name", 
            "local_authority_name", "region_name", "country_name"]]

Pandas also has a nice feature where you can specify a range of columns using the names of the first and last column

In [ ]:
approximated_social_grade_stage = approximated_social_grade.loc[:,
       "geography_code":"c2_skilled_manual_occupations"]
approximated_social_grade_stage.head()

### Where

Filtering can be done in pandas by placing a logical statment inside square brackets.

In [ ]:
england_data = admin_stage[admin_stage['country_name'] == "England"]
england_data.head()

As with the selects we can also use `loc`.

In [ ]:
england_data = admin_stage.loc[admin_stage['country_name'] == "England",:]

As you've probably guessed the real advantage of `loc` is that we can combine the select and filtering operations.

In [ ]:
england_data = admin_stage.loc[admin_stage['country_name'] == "England",["local_authority_name", "region_name"]]
england_data.head()

### Order by

Here we simply use the `sort_values` method.

In [ ]:
england_sorted = england_data.sort_values(by=['region_name', 'local_authority_name'])
england_sorted.head()

Note how we can *chain* these methods to do many jobs in one go.

In [ ]:
england_sorted = admin_stage.loc[admin_stage['country_name'] == "England",["local_authority_name", "region_name"]].sort_values(by=['region_name', 'local_authority_name'])
england_sorted.head()

### Joins

The `merge` function covers the full range of SQL joins.

In [ ]:
customer_add_ward_code = pd.merge(customer_data, 
       postcode_to_ward_code, how="inner", on = "postcode")
customer_add_ward_code.head()

Note we need to drop some columns which is easily done.

In [ ]:
customer_add_ward_code = customer_add_ward_code.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'])
customer_add_ward_code.head()

### Group by

Grouping and summarising is a little different. Pandas uses a concept called **hierarchical indexing**. This is worth looking up as it is the source of a lot of confusion if you are coming at pandas from R or SQL. For now we should note that instead of using separate columns to describe the values in a summary table, it gives them something akin to row and column labels. This makes some things very easy but if you were expecting, and need, columns not labels then it can be inconvenient. Fortunately there is a parameter (`as_index=False`) which will give us the output we want. 
  
Compare output with and without the `as_index=False` setting.
  
With:

In [ ]:
sum_table = customer_add_ward_code.loc[:,["age", "one_day_hours", 
                "seven_day_hours"]].groupby('age', as_index=False).sum()
sum_table.head()

And without:

In [ ]:
sum_table = customer_add_ward_code.loc[:,["age", "one_day_hours", 
                "seven_day_hours"]].groupby('age').sum()
sum_table.head()

### Creating new variables

To do this we use the `assign` method. For example here we create an age gender interaction variable.

In [ ]:
customer_add_ward_code = customer_add_ward_code.assign(age_gender = customer_add_ward_code.age * customer_add_ward_code.gender)
customer_add_ward_code.head()

### Renaming variables

This is straightforward.

In [ ]:
customer_add_ward_code = customer_add_ward_code.rename(columns={'affluent': 'affluence'})
customer_add_ward_code.head()

## Your task

Now you have everything you need to augment the customer data with the census data. You will need to 

1. Take each census table and select some useful columns, if necessary renaming them as something more usable
2. Create within these tables some new penetration variables (for example if we are working on the *approximated_social_grade* table we could create a variable called `pct_c1_supervisory`)
3. Join this new data to `customer_add_ward_code` using the `ward_code` (it's called `geography_code` on many of the tables)
4. Make sure you also attach the useful geographical information in the `admin_regions` table.

When this is complete, repeat with your own viewing data!
